# Getting U2M data from NASA

In [1]:
import os
import re
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from calendar import monthrange
from opendap_download.multi_processing_download import DownloadManager

In [2]:
####### INPUTS - CHANGE THESE #########
username = 'bhauma' # Username for MERRA download account
password = 'Ankita@0123' # Password for MERRA download account

In [3]:
years = [2020] # List of years for which data will be downloaded
field_id = 'U2M' # ID of field in MERRA-2 - find ID here: https://gmao.gsfc.nasa.gov/pubs/docs/Bosilovich785.pdf 
field_name = '2M Eastward Wind' # Name of field to be stored with downloaded data (can use any name you like)
database_name = 'M2TMNXSLV' # Name of database in which field is stored, can be looked up by ID here: 
database_id = 'tavgM_2d_slv_Nx'

In [4]:
df = pd.read_csv("../Substation/geo_locations/geo_location_NC.csv")
df['name'] = 'North Carolina'

In [5]:
locs = df.values.tolist()
len(locs)

2141

In [6]:
aggregator = 'max' # Method by which data will be aggregated over days and weeks. Can be "sum", "mean", "min", or "max" 

In [7]:
lat_coords = np.arange(0, 361, dtype=int)
lon_coords = np.arange(0, 576, dtype=int)
#database_url = 'https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/MERRA2_MONTHLY/' + database_name + '.5.12.4/'
database_url = 'https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/MERRA2_MONTHLY/M2TMNXSLV.5.12.4/'
NUMBER_OF_CONNECTIONS = 5

In [8]:
####### DOWNLOAD DATA #########
# Translate lat/lon into coordinates that MERRA-2 understands
def translate_lat_to_geos5_native(latitude):
    """
    The source for this formula is in the MERRA2 
    Variable Details - File specifications for GEOS pdf file.
    The Grid in the documentation has points from 1 to 361 and 1 to 576.
    The MERRA-2 Portal uses 0 to 360 and 0 to 575.
    latitude: float Needs +/- instead of N/S
    """
    return ((latitude + 90) / 0.5)

In [9]:
def translate_lon_to_geos5_native(longitude):
    """See function above"""
    return ((longitude + 180) / 0.625)

In [10]:
def find_closest_coordinate(calc_coord, coord_array):
    """
    Since the resolution of the grid is 0.5 x 0.625, the 'real world'
    coordinates will not be matched 100% correctly. This function matches 
    the coordinates as close as possible. 
    """
    # np.argmin() finds the smallest value in an array and returns its
    # index. np.abs() returns the absolute value of each item of an array.
    # To summarize, the function finds the difference closest to 0 and returns 
    # its index. 
    index = np.abs(coord_array-calc_coord).argmin()
    return coord_array[index]

In [11]:
def translate_year_to_file_number(year):
    """
    The file names consist of a number and a meta data string. 
    The number changes over the years. 1980 until 1991 it is 100, 
    1992 until 2000 it is 200, 2001 until 2010 it is  300 
    and from 2011 until now it is 400.
    """
    file_number = ''
    
    if year >= 1980 and year < 1992:
        file_number = '100'
    elif year >= 1992 and year < 2001:
        file_number = '200'
    elif year >= 2001 and year < 2011:
        file_number = '300'
    elif year >= 2011:
        file_number = '400'
    else:
        raise Exception('The specified year is out of range.')
    return file_number

In [12]:
def generate_url_params(parameter, time_para, lat_para, lon_para):
    """Creates a string containing all the parameters in query form"""
    parameter = map(lambda x: x + time_para, parameter)
    parameter = map(lambda x: x + lat_para, parameter)
    parameter = map(lambda x: x + lon_para, parameter)
    return ','.join(parameter)

In [13]:
def generate_download_links(download_years, base_url, dataset_name, url_params):
    """
    Generates the links for the download. 
    download_years: The years you want to download as array. 
    dataset_name: The name of the data set. For example tavg1_2d_slv_Nx
    """
    urls = []
    for y in download_years: 
        y_str = str(y)
        file_num = translate_year_to_file_number(y)
        for m in range(1,13):
            m_str = str(m).zfill(2)
            # Create the file name string
            file_name = 'MERRA2_{num}.{name}.{y}{m}.nc4'.format(
                num=file_num, name=dataset_name, 
                y=y_str, m=m_str)
            # Create the query
            query = '{base}{y}/{name}.nc4?{params}'.format(
                base=base_url, y=y_str,
                name=file_name, params=url_params)
            urls.append(query)
    return urls

In [14]:
print('DOWNLOADING DATA FROM MERRA')
print('Predicted time: ' + str(len(years)*len(locs)*6) + ' minutes')
print('=====================')
for lat, lon, loc in locs:
    print('Downloading ' + field_name + ' data for ' + loc + str(lat) + str(lon))
    # Translate the coordinates that define your area to grid coordinates.
    lat_coord = translate_lat_to_geos5_native(lat)
    lon_coord = translate_lon_to_geos5_native(lon)
    # Find the closest coordinate in the grid.
    lat_closest = find_closest_coordinate(lat_coord, lat_coords)
    lon_closest = find_closest_coordinate(lon_coord, lon_coords)
    # Generate URLs for scraping
    requested_lat = '[{lat}:1:{lat}]'.format(lat=lat_closest)
    requested_lon = '[{lon}:1:{lon}]'.format(lon=lon_closest)
    parameter = generate_url_params([field_id], '[0:1:0]', requested_lat, requested_lon)
    generated_URL = generate_download_links(years, database_url, database_id, parameter)
    download_manager = DownloadManager()
    download_manager.set_username_and_password(username, password)
    download_manager.download_path = field_name + '/' + loc
    download_manager.download_urls = generated_URL
    %time download_manager.start_download(NUMBER_OF_CONNECTIONS)

DOWNLOADING DATA FROM MERRA
Predicted time: 12846 minutes
CPU times: user 483 ms, sys: 103 ms, total: 586 ms
Wall time: 4.67 s
CPU times: user 525 ms, sys: 66.7 ms, total: 592 ms
Wall time: 4.33 s
CPU times: user 521 ms, sys: 82.6 ms, total: 603 ms
Wall time: 4.24 s
CPU times: user 563 ms, sys: 77.2 ms, total: 640 ms
Wall time: 4.41 s
CPU times: user 406 ms, sys: 61.3 ms, total: 467 ms
Wall time: 4.19 s
CPU times: user 549 ms, sys: 81.6 ms, total: 631 ms
Wall time: 4.39 s
CPU times: user 490 ms, sys: 64.4 ms, total: 555 ms
Wall time: 4.3 s
CPU times: user 499 ms, sys: 67.2 ms, total: 566 ms
Wall time: 4.39 s
CPU times: user 568 ms, sys: 56.3 ms, total: 624 ms
Wall time: 4.36 s
CPU times: user 518 ms, sys: 64 ms, total: 582 ms
Wall time: 4.27 s
CPU times: user 437 ms, sys: 57.6 ms, total: 494 ms
Wall time: 4.3 s
CPU times: user 507 ms, sys: 73.2 ms, total: 580 ms
Wall time: 4.19 s
CPU times: user 539 ms, sys: 56.7 ms, total: 596 ms
Wall time: 4.31 s
CPU times: user 490 ms, sys: 63 ms, t

CPU times: user 506 ms, sys: 53.1 ms, total: 559 ms
Wall time: 4.22 s
CPU times: user 477 ms, sys: 64.1 ms, total: 541 ms
Wall time: 4.23 s
CPU times: user 532 ms, sys: 62.1 ms, total: 595 ms
Wall time: 4.45 s
CPU times: user 547 ms, sys: 57.1 ms, total: 604 ms
Wall time: 4.46 s
CPU times: user 525 ms, sys: 58.3 ms, total: 583 ms
Wall time: 4.39 s
CPU times: user 526 ms, sys: 44.5 ms, total: 570 ms
Wall time: 4.31 s
CPU times: user 509 ms, sys: 46.3 ms, total: 555 ms
Wall time: 4.25 s
CPU times: user 548 ms, sys: 69.4 ms, total: 617 ms
Wall time: 4.34 s
CPU times: user 571 ms, sys: 49.4 ms, total: 620 ms
Wall time: 4.35 s
CPU times: user 603 ms, sys: 71 ms, total: 674 ms
Wall time: 4.61 s
CPU times: user 593 ms, sys: 71.3 ms, total: 664 ms
Wall time: 4.83 s
CPU times: user 520 ms, sys: 69 ms, total: 589 ms
Wall time: 4.39 s
CPU times: user 558 ms, sys: 62.2 ms, total: 621 ms
Wall time: 4.22 s
CPU times: user 557 ms, sys: 66.8 ms, total: 624 ms
Wall time: 4.41 s
CPU times: user 558 ms, 

CPU times: user 543 ms, sys: 50.2 ms, total: 594 ms
Wall time: 4.41 s
CPU times: user 603 ms, sys: 86.5 ms, total: 689 ms
Wall time: 4.34 s
CPU times: user 547 ms, sys: 60.2 ms, total: 607 ms
Wall time: 4.42 s
CPU times: user 487 ms, sys: 57.3 ms, total: 544 ms
Wall time: 5.72 s
CPU times: user 486 ms, sys: 71.8 ms, total: 558 ms
Wall time: 4.29 s
CPU times: user 489 ms, sys: 63.7 ms, total: 553 ms
Wall time: 4.37 s
CPU times: user 455 ms, sys: 62.7 ms, total: 517 ms
Wall time: 4.27 s
CPU times: user 483 ms, sys: 69.3 ms, total: 552 ms
Wall time: 4.28 s
CPU times: user 432 ms, sys: 47.7 ms, total: 479 ms
Wall time: 4.18 s
CPU times: user 500 ms, sys: 53.5 ms, total: 554 ms
Wall time: 4.33 s
CPU times: user 494 ms, sys: 45 ms, total: 539 ms
Wall time: 4.19 s
CPU times: user 496 ms, sys: 78.4 ms, total: 574 ms
Wall time: 4.22 s
CPU times: user 535 ms, sys: 50.7 ms, total: 586 ms
Wall time: 4.23 s
CPU times: user 493 ms, sys: 53 ms, total: 546 ms
Wall time: 4.24 s
CPU times: user 500 ms, 

CPU times: user 506 ms, sys: 53.9 ms, total: 560 ms
Wall time: 4.48 s
CPU times: user 537 ms, sys: 71.5 ms, total: 609 ms
Wall time: 4.29 s
CPU times: user 552 ms, sys: 54 ms, total: 606 ms
Wall time: 4.42 s
CPU times: user 503 ms, sys: 73.7 ms, total: 577 ms
Wall time: 4.46 s
CPU times: user 548 ms, sys: 62.7 ms, total: 611 ms
Wall time: 4.45 s
CPU times: user 515 ms, sys: 52.2 ms, total: 567 ms
Wall time: 4.29 s
CPU times: user 515 ms, sys: 68.4 ms, total: 583 ms
Wall time: 4.28 s
CPU times: user 518 ms, sys: 53.8 ms, total: 572 ms
Wall time: 4.28 s
CPU times: user 476 ms, sys: 51.5 ms, total: 528 ms
Wall time: 4.35 s
CPU times: user 527 ms, sys: 68.3 ms, total: 596 ms
Wall time: 4.37 s
CPU times: user 516 ms, sys: 46.7 ms, total: 563 ms
Wall time: 4.28 s
CPU times: user 513 ms, sys: 64.7 ms, total: 578 ms
Wall time: 4.34 s
CPU times: user 476 ms, sys: 56.5 ms, total: 533 ms
Wall time: 4.3 s
CPU times: user 482 ms, sys: 55.8 ms, total: 538 ms
Wall time: 4.6 s
CPU times: user 518 ms, 

CPU times: user 505 ms, sys: 69.4 ms, total: 574 ms
Wall time: 4.64 s
CPU times: user 504 ms, sys: 56.1 ms, total: 560 ms
Wall time: 4.26 s
CPU times: user 480 ms, sys: 51.7 ms, total: 531 ms
Wall time: 4.18 s
CPU times: user 486 ms, sys: 66.1 ms, total: 552 ms
Wall time: 4.37 s
CPU times: user 511 ms, sys: 61 ms, total: 572 ms
Wall time: 4.29 s
CPU times: user 433 ms, sys: 49.2 ms, total: 482 ms
Wall time: 4.29 s
CPU times: user 464 ms, sys: 54.3 ms, total: 519 ms
Wall time: 4.26 s
CPU times: user 434 ms, sys: 49.9 ms, total: 484 ms
Wall time: 4.24 s
CPU times: user 453 ms, sys: 66.9 ms, total: 519 ms
Wall time: 4.33 s
CPU times: user 468 ms, sys: 52.2 ms, total: 520 ms
Wall time: 4.18 s
CPU times: user 490 ms, sys: 66.5 ms, total: 557 ms
Wall time: 4.27 s
CPU times: user 440 ms, sys: 53.5 ms, total: 494 ms
Wall time: 4.31 s
CPU times: user 428 ms, sys: 47.4 ms, total: 475 ms
Wall time: 4.47 s
CPU times: user 432 ms, sys: 46.5 ms, total: 478 ms
Wall time: 4.15 s
CPU times: user 473 ms

...

CPU times: user 585 ms, sys: 69 ms, total: 654 ms
Wall time: 5.28 s
CPU times: user 572 ms, sys: 55 ms, total: 627 ms
Wall time: 5.24 s
CPU times: user 574 ms, sys: 67.9 ms, total: 642 ms
Wall time: 5.26 s
CPU times: user 566 ms, sys: 68.4 ms, total: 634 ms
Wall time: 5.45 s
CPU times: user 576 ms, sys: 74.8 ms, total: 650 ms
Wall time: 5.27 s
CPU times: user 558 ms, sys: 55.4 ms, total: 613 ms
Wall time: 5.07 s
CPU times: user 570 ms, sys: 52.1 ms, total: 622 ms
Wall time: 5.52 s
CPU times: user 588 ms, sys: 95.4 ms, total: 684 ms
Wall time: 5.33 s
CPU times: user 574 ms, sys: 57.3 ms, total: 632 ms
Wall time: 4.37 s
CPU times: user 561 ms, sys: 61.7 ms, total: 622 ms
Wall time: 4.77 s
CPU times: user 552 ms, sys: 61.9 ms, total: 614 ms
Wall time: 4.5 s
CPU times: user 579 ms, sys: 55.2 ms, total: 634 ms
Wall time: 4.28 s
CPU times: user 599 ms, sys: 65.8 ms, total: 665 ms
Wall time: 4.41 s
CPU times: user 579 ms, sys: 68.7 ms, total: 648 ms
Wall time: 4.38 s
CPU times: user 563 ms, s

CPU times: user 560 ms, sys: 58.3 ms, total: 618 ms
Wall time: 4.39 s
CPU times: user 502 ms, sys: 41 ms, total: 543 ms
Wall time: 4.34 s
CPU times: user 578 ms, sys: 54.4 ms, total: 633 ms
Wall time: 4.62 s
CPU times: user 511 ms, sys: 46.4 ms, total: 557 ms
Wall time: 4.51 s
CPU times: user 513 ms, sys: 43.6 ms, total: 557 ms
Wall time: 4.68 s
CPU times: user 511 ms, sys: 48.6 ms, total: 559 ms
Wall time: 4.4 s
CPU times: user 517 ms, sys: 54.4 ms, total: 571 ms
Wall time: 4.24 s
CPU times: user 505 ms, sys: 52.2 ms, total: 557 ms
Wall time: 4.5 s
CPU times: user 523 ms, sys: 44.7 ms, total: 567 ms
Wall time: 4.6 s
CPU times: user 527 ms, sys: 47.3 ms, total: 574 ms
Wall time: 4.53 s
CPU times: user 495 ms, sys: 46.2 ms, total: 542 ms
Wall time: 4.45 s
CPU times: user 564 ms, sys: 53.8 ms, total: 618 ms
Wall time: 4.3 s
CPU times: user 559 ms, sys: 54.4 ms, total: 614 ms
Wall time: 4.68 s
CPU times: user 607 ms, sys: 70.4 ms, total: 677 ms
Wall time: 4.37 s
CPU times: user 582 ms, sy

CPU times: user 520 ms, sys: 47.9 ms, total: 568 ms
Wall time: 4.28 s
CPU times: user 584 ms, sys: 76.7 ms, total: 661 ms
Wall time: 4.45 s
CPU times: user 566 ms, sys: 75.4 ms, total: 642 ms
Wall time: 4.6 s
CPU times: user 573 ms, sys: 81.2 ms, total: 654 ms
Wall time: 4.38 s
CPU times: user 585 ms, sys: 85 ms, total: 670 ms
Wall time: 4.39 s
CPU times: user 562 ms, sys: 70.5 ms, total: 633 ms
Wall time: 4.4 s
CPU times: user 533 ms, sys: 63.6 ms, total: 597 ms
Wall time: 4.35 s
CPU times: user 544 ms, sys: 61.6 ms, total: 606 ms
Wall time: 4.38 s
CPU times: user 551 ms, sys: 80.8 ms, total: 632 ms
Wall time: 4.39 s
CPU times: user 575 ms, sys: 63.6 ms, total: 639 ms
Wall time: 4.27 s
CPU times: user 595 ms, sys: 70.3 ms, total: 665 ms
Wall time: 4.39 s
CPU times: user 574 ms, sys: 58.6 ms, total: 633 ms
Wall time: 4.39 s
CPU times: user 580 ms, sys: 64 ms, total: 644 ms
Wall time: 4.39 s
CPU times: user 568 ms, sys: 77.3 ms, total: 645 ms
Wall time: 4.41 s
CPU times: user 553 ms, sy

CPU times: user 493 ms, sys: 48.4 ms, total: 541 ms
Wall time: 4.26 s
CPU times: user 584 ms, sys: 78.3 ms, total: 662 ms
Wall time: 4.71 s
CPU times: user 588 ms, sys: 71.2 ms, total: 660 ms
Wall time: 4.46 s
CPU times: user 529 ms, sys: 48.3 ms, total: 578 ms
Wall time: 4.41 s
CPU times: user 524 ms, sys: 57.4 ms, total: 581 ms
Wall time: 4.29 s
CPU times: user 532 ms, sys: 52 ms, total: 584 ms
Wall time: 4.25 s
CPU times: user 551 ms, sys: 59.5 ms, total: 611 ms
Wall time: 4.31 s
CPU times: user 546 ms, sys: 64.4 ms, total: 610 ms
Wall time: 4.58 s
CPU times: user 495 ms, sys: 57.3 ms, total: 552 ms
Wall time: 4.54 s
CPU times: user 524 ms, sys: 47.9 ms, total: 572 ms
Wall time: 4.34 s
CPU times: user 545 ms, sys: 55.1 ms, total: 601 ms
Wall time: 4.5 s
CPU times: user 566 ms, sys: 56 ms, total: 622 ms
Wall time: 4.46 s
CPU times: user 485 ms, sys: 55.4 ms, total: 540 ms
Wall time: 4.39 s
CPU times: user 524 ms, sys: 54.4 ms, total: 579 ms
Wall time: 4.65 s
CPU times: user 563 ms, s

CPU times: user 560 ms, sys: 47.4 ms, total: 607 ms
Wall time: 4.17 s
CPU times: user 573 ms, sys: 73.4 ms, total: 646 ms
Wall time: 4.29 s
CPU times: user 553 ms, sys: 51.8 ms, total: 605 ms
Wall time: 4.23 s
CPU times: user 464 ms, sys: 55 ms, total: 519 ms
Wall time: 4.16 s
CPU times: user 575 ms, sys: 60.7 ms, total: 636 ms
Wall time: 4.34 s
CPU times: user 584 ms, sys: 58.9 ms, total: 643 ms
Wall time: 4.33 s
CPU times: user 577 ms, sys: 61.2 ms, total: 638 ms
Wall time: 4.25 s
CPU times: user 549 ms, sys: 44.3 ms, total: 594 ms
Wall time: 4.16 s
CPU times: user 588 ms, sys: 50 ms, total: 638 ms
Wall time: 4.31 s
CPU times: user 558 ms, sys: 67.6 ms, total: 625 ms
Wall time: 4.28 s
CPU times: user 587 ms, sys: 60.5 ms, total: 647 ms
Wall time: 4.25 s
CPU times: user 585 ms, sys: 70.6 ms, total: 655 ms
Wall time: 4.35 s
CPU times: user 539 ms, sys: 51.8 ms, total: 591 ms
Wall time: 4.16 s
CPU times: user 566 ms, sys: 44.4 ms, total: 610 ms
Wall time: 4.22 s
CPU times: user 589 ms, 

In [15]:
######### OPEN, CLEAN, MERGE, MERGE DATA AND WRITE CSVS ##########
def extract_date(data_set):
    """
    Extracts the date from the filename before merging the datasets. 
    """ 
    if 'HDF5_GLOBAL.Filename' in data_set.attrs:
        f_name = data_set.attrs['HDF5_GLOBAL.Filename']
    elif 'Filename' in data_set.attrs:
        f_name = data_set.attrs['Filename']
    else: 
        raise AttributeError('The attribute name has changed again!')
    # find a match between "." and ".nc4" that does not have "." .
    exp = r'(?<=\.)[^\.]*(?=\.nc4)'
    res = re.search(exp, f_name).group(0)
    # Extract the date. 
    y, m = res[0:4], res[4:6]
    date_str = ('%s-%s' % (y, m))
    data_set = data_set.assign(date=date_str)
    return data_set

In [16]:
# Open nc4 files as dataframes, perform aggregations and save as CSV files
print('CLEANING AND MERGING DATA')
print('Predicted time: ' + str(len(years)*len(locs)*0.1) + ' minutes')
print('=====================')

lats = []
lons = []
dates = []
u2ms = []

for lat, lon, loc in locs:
    print('Cleaning and merging ' + field_name + ' data for ' + loc + str(lat) + str(lon))
    for file in os.listdir(field_name + '/' + loc):
        if '.nc4' in file:
            try:
                with xr.open_mfdataset(field_name + '/' + loc + '/' + file, preprocess=extract_date) as df:
                    u2ms.append(df.to_dataframe()["U2M"].values[0])
                    dates.append(df.to_dataframe()["date"].values[0])
                    lats.append(lat)
                    lons.append(lon)
            except:
                print('Issue with file ' + file)
    
df_monthly = pd.DataFrame({"date": dates, "lat": lats, "lon": lons, "U2M": u2ms})
df_monthly.to_csv(field_name + '/' + loc + '_monthly.csv', index = False)

print('FINISHED')

CLEANING AND MERGING DATA
Predicted time: 214.10000000000002 minutes
Cleaning and merging 2M Eastward Wind data for North Carolina-84.13043898335.026828514
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-84.01945538535.0761326630001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-84.177243825999935.1502678150001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-82.84279063435.5379700560001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-83.79140801435.0190416460001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-83.07507299935.242909323
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-83.

Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-82.63856804435.5392498080001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-82.544648305999935.4716652930001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-82.54184962735.4725411790001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-82.606076631999935.515989586
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-82.52518958535.4790835740001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-82.486988510999935.54417254
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-82.49207220

Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.88498874536.161466497
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.924346130999936.0681032740001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.88235168836.066428923
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.65804857536.2351899780001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.70704030936.2173090140001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.758473013999936.263987909
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.65880320936.204

Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.15650490435.917043092
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.07759523535.887598827
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.052350676999935.8645868950001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.08503401635.758032805
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.110128926999936.1779437120001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-80.92402520936.2298466030001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-80.9501944636.2247

Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.88506576235.3408076870001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.866764422999935.332065098
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.85509880535.307969449
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.33163360135.766310015
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.284644635999935.752052178
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.260293159999935.73094194
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.26037397135.73045186

...

Cleaning and merging 2M Eastward Wind data for North Carolina-81.68099983335.474757283
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.68065719135.4753426300001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.50105087935.3854213280001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.456046280999935.35804663
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.36584707635.3219319930001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.613738882999935.3147904950001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.35240636535.21850935
Issue with file MERRA2_400.tavgM_2d_slv_Nx.20

Cleaning and merging 2M Eastward Wind data for North Carolina-81.25365012635.188089662
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.08578684135.1689335450001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.361981119999935.385532288
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.06617720735.2491709230001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.13683399535.3727015310001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.264673565999935.4270180070001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.12024638535.2400596640001
Issue with file MERRA2_400.tavgM_2d_slv

Cleaning and merging 2M Eastward Wind data for North Carolina-81.98171518735.688349959
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-82.02161995835.795121841
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-82.059877840999935.6915552400001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-82.06697101435.683148605
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.96983125135.5498896450001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-80.881615581999935.0765846080001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-80.87649974135.0643827320001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.

Cleaning and merging 2M Eastward Wind data for North Carolina-79.890637695999936.4670456490001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-79.890471133999936.4683127120001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-79.8885913636.4692347540001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-79.7862315536.509010377
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-79.78567832636.5111578400001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-79.76345716636.4868258710001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-79.76043049236.4873493530001
Issue with file MERRA2_400.tavgM_2d_s

Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-77.545953999999935.9233390000001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-80.26824136.091323
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-79.756504999999936.080206
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-80.82786335.564245
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-79.17639735.547173
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-79.18825735.549738
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Eastward Wind data for North Carolina-81.986736999999935.69886
Issue with file MERRA2_400.t